## asdf

In [1]:
from twoplayer import make_pdf, construct_instance_from_dict, \
    store_instance_in_db, load_db, select_instance_dict_from_db, log

In [2]:
from sudokugen import instances, generate_puzzle, encodings, masks

In [3]:
from copy import deepcopy
import json
import random
import textwrap
from datetime import datetime

In [4]:
database = load_db("building-blocks-i.json")

def select_puzzle(category, puzzle_id):
    return construct_instance_from_dict(
        select_instance_dict_from_db(
            database,
            category,
            puzzle_id
        )
    )

puzzle_choice = {
    ("A", 1): ("1a", 1),
    ("A", 2): ("2a", 1),
    ("A", 3): ("3a", 1),
    ("B", 1): ("2a", 4),
    ("B", 2): ("3a", 3),
    ("B", 3): ("4a", 1),
}

instance_dict = {}
for player in ["A", "B"]:
    for puzzle_no in [1, 2, 3]:
        category, puzzle_id = puzzle_choice[(player, puzzle_no)]
        instance_dict[(player, puzzle_no)] = select_puzzle(category, puzzle_id)

meta_info = {
    "level": {
        (player, puzzle_no): int(puzzle_choice[(player, puzzle_no)][0][0])
        for player in ["A", "B"]
        for puzzle_no in [1, 2, 3]
    },
    "deco": {
        "A": {
            "input color": "red",
            "input pattern": "north west",
            "output color": "blue",
            "output pattern": "horizontal",
        },
        "B": {
            "input color": "blue",
            "input pattern": "horizontal",
            "output color": "red",
            "output pattern": "north west",
        },
    },
}

In [5]:
arranged_instance_dict = {}

# Randomly construct a value permutation
# that we will use for all instances
value_list = list(range(1,10))
random.shuffle(value_list)

for player in ["A", "B"]:
    for puzzle_no in [1, 2, 3]:
        
        instance = deepcopy(instance_dict[(player, puzzle_no)])
            
        # For player B, swap the input and output solutions and decoy values
        if player == "B":
            instance.swap_values(1, 2)
            instance.swap_values(3, 4)

        # Apply the value permutation
        instance.apply_value_permutation(value_list)

        # Shuffle the orientation of the puzzle
        instance.shuffle_orientation()
        
        arranged_instance_dict[(player, puzzle_no)] = instance

In [6]:
filename="puzzle-0-1"
puzzle_set_id="0-1"

make_pdf(
    filename=filename,
    directory="output",
    instance_dict=arranged_instance_dict,
    meta_info=meta_info,
    puzzle_set_id=puzzle_set_id,
)

# Logging
logstr = datetime.now().strftime("=== %d-%m-%Y %H:%M:%S ===\n")
logstr += f"- Created {filename}.pdf\n"
logstr += f"- Puzzle set id: {puzzle_id}\n"
for player in ["A", "B"]:
    for puzzle_no in [1, 2, 3]:
        category, puzzle_id = puzzle_choice[(player, puzzle_no)]
        logstr += f"- Puzzle #{puzzle_no} for {player}: {category}, {puzzle_id}\n"

logstr += f"- Value permutation used: {value_list}\n"

log(logstr)